# Le jeu du Bâton

In [112]:
class Game(object):
    def __init__(self, base):
        self.base = base
        self.state = [[True]*i for i in range(1,base+1)]
        self.solved = False
        self.n_iter = 0
        
    def _check_valid_move(self, move):
        assert(all(self.state[move[0]][move[1][0]:move[1][1]+1])) 
    
    def next_move(self, move):
        if not(self.solved): 
            try:
                self._check_valid_move(move)
                self.state[move[0]][move[1][0]:move[1][1]+1] = [False for i in range(move[1][0],move[1][1]+1)]
                self.n_iter += 1
                if all([not elt for i in range(len(self.state)) for elt in self.state[i]]): 
                    self.solved = True
                    
            except: 
                print('Invalid move, please try again...')
        else: 
            print('Game finished!')


In [113]:
class Agent(object): 
    def decision(self,state): 
        return

In [2]:
class Node():
    def __init__(self, state):
        self.state = state

In [ ]:
class Tree(object):
    "Generic tree node."
    def __init__(self, move = 'root', value = 1, children=None):
        self.move = move
        self.children = []
        self.value = value
        if children is not None:
            for child in children:
                self.add_child(child)
                
    def __repr__(self):
        return self.value
    
    def add_child(self, node):
        assert isinstance(node, Tree)
        self.children.append(node)
           

In [ ]:
class AgentMCTS(Agent):
    def __init__(self):
        self.tree = Tree()

In [15]:
from random import choice
from monte_carlo_tree_search import MCTS, Node

# Inheriting from a namedtuple is convenient because it makes the class
# immutable and predefines __init__, __repr__, __hash__, __eq__, and others
class MarienbadBoard(Node):
    def __init__(self, base):
        self.base = base
        self.state = [[True]*i for i in range(1,base+1)]
        self.solved = False
        self.winner = None #None si pas encore fini, True si joueur, False si machine
        self.n_iter = 0
        self.turn = True #True si c'est au joueur de jouer, False si machine
    
    def is_terminal(self):
        return self.solved
        
    def find_children(self):
        if self.solved:  # If the game is finished then no moves can be made
            return set()
        # Otherwise, you can make a move in each of the empty spots
        moves = []
        for ind in range(self.base):
            for ind2 in range(ind+1):
                for ind3 in range(min(3, ind+1-ind2)):
                    if sum(self.state[ind][ind2:ind2+ind3]) == 0:
                        moves.append([ind1, [ind2, ind2 + ind3]])
        return {
            self.next_move(move) for move in moves
#             board.make_move(i) for i, value in enumerate(board.tup) if value is None
        }

    def find_random_child(self):
        if self.solved:
            return None  # If the game is finished then no moves can be made
        moves = []
        for ind in range(self.base):
            for ind2 in range(ind+1):
                for ind3 in range(min(3, ind+1-ind2)):
                    if sum(self.state[ind][ind2:ind2+ind3]) == 0:
                        moves.append([ind1, [ind2, ind2 + ind3]])
        return self.next_move(choice(moves))
    
    def _check_valid_move(self, move):
        assert(all(self.state[move[0]][move[1][0]:move[1][1]+1])) 

    def reward(self):
        if not self.solved:
            raise RuntimeError(f"reward called on nonterminal board {board}")
#         if board.winner is board.turn:
#             # It's your turn and you've already won. Should be impossible.
#             raise RuntimeError(f"reward called on unreachable board {board}")
        if self.winner:
            return 0  # Your opponent has just won. Bad.
        # The winner is neither True or False
        raise RuntimeError(f"board has unknown winner type {board.winner}")

    def next_move(self, move):
        if not(self.solved): 
            try:
                self._check_valid_move(move)
                self.state[move[0]][move[1][0]:move[1][1]+1] = [False for i in range(move[1][0],move[1][1]+1)]
                self.n_iter += 1
                self.turn = not self.turn
                winner = self.find_winner()
            except: 
                print('Invalid move, please try again...')
        else: 
            print('Game finished!')
    
    def find_winner(self):
        if all([not elt for i in range(len(self.state)) for elt in self.state[i]]): 
            self.solved = True
            return (not self.turn)
        return None

    def to_pretty_string(self):
        to_char = lambda v: ("I" if v is True else "X")
        rows = [
            [to_char(el) for el in self.state[ind]] for ind in range(self.base)
        ]
        return rows
    
    def __hash__(self):
        "Nodes must be hashable"
        return 123456789

    def __eq__(node1, node2):
        "Nodes must be comparable"
        return True


def play_game():
    tree = MCTS()
    board = new_marienbad(6)
    print(board.to_pretty_string())
    while True:
        row = input('enter row: ')
        row = int(row)
        cols = input("enter cols: ")
        cols = cols.split(",")
        move = [row, cols]
        board._check_valid_move(move)
        board.next_move(move)
        print(board.to_pretty_string())
        if board.is_terminal():
            break
        # You can train as you go, or only at the beginning.
        # Here, we train as we go, doing fifty rollouts each turn.
        for _ in range(50):
            tree.do_rollout(board)
        board = tree.choose(board)
        print(board.to_pretty_string())
        if board.is_terminal():
            break


def new_marienbad(base):
    return MarienbadBoard(base)


if __name__ == "__main__":
    play_game()

[['I'], ['I', 'I'], ['I', 'I', 'I'], ['I', 'I', 'I', 'I'], ['I', 'I', 'I', 'I', 'I'], ['I', 'I', 'I', 'I', 'I', 'I']]
enter row: 1
enter cols: 1,2


TypeError: can only concatenate str (not "int") to str